In [1]:
import tkinter as tk
from tkinter import Label
from tkinter import filedialog
import cv2
import mediapipe as mp
from PIL import Image, ImageTk

In [3]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

In [5]:
cap = None
init_y = None
min_y = 1.0
running = False

def frame_pro(frame):
    global init_y, min_y
    color_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(color_frame)

    if results.pose_landmarks:
        hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
        y = hip.y

        if init_y is None:
            init_y = y
        else:
            min_y = min(min_y, y)
        mp.solutions.drawing_utils.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
    return frame

In [7]:
def video():
    global cap, running

    if cap.isOpened():
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (640, 480))
            frame = frame_pro(frame)
            img= cv2. cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(img)
            imgtk = ImageTk.PhotoImage(image=img)

            video_label.imgtk = imgtk
            video_label.configure(image=imgtk)
    if running:
        video_label.after(10, video)
    else:
        result()

In [9]:
def realtime():
    global cap, running, init_y, min_y
    reset()
    cap = cv2.VideoCapture(0)
    running = True
    video()

def upload_video():
    global cap, running, init_y, min_y
    reset()
    video_path = filedialog.askopenfilename(filetypes = [("Video Files", "*.mp4 *.avi")])
    if video_path:
        cap = cv2.VideoCapture(video_path)
        running = True
        video()

In [11]:
def stop_video():
    global running
    running = False
    if cap:
        cap.release()

def reset():
    global init_y, min_y
    init_y = None
    min_y = 1.0
    result_label.config(text="")
    video_label.config(image="")

In [13]:
def result():
    global init_y, min_y
    if init_y is not None and min_y is not None:
        jump_height = init_y - min_y
        result_label.config(text=f"Jump Height : {jump_height:.3f}")

In [15]:
root = tk.Tk()
root.title("Jump Height Estimator")
root.geometry("400x350")

heading = Label(root,text="Jump Height Estimator",font=("Times New Roman", 24, "bold"),justify="center",fg="saddle brown")
heading.pack(pady=10) 

btn_realtime = tk.Button(root, text="Real-Time Video", command=realtime, width=30, height= 3)
btn_realtime.pack(pady=10)

btn_upload = tk.Button(root, text="Video Upload", command=upload_video, width=30, height= 3)
btn_upload.pack(pady=10)

btn_frame = tk.Frame(root)
btn_frame.pack(pady=10)

btn_stop = tk.Button(btn_frame, text="Stop", command=stop_video, width=15, height= 2)
btn_stop.pack(side="left", padx=5)

btn_reset = tk.Button(btn_frame, text="Reset", command=reset, width=15, height= 2)
btn_reset.pack(side="left", padx=5)

video_label = tk.Label(root)
video_label.pack()

result_label = tk.Label(root, text="", font=("Times New Roman", 16))
result_label.pack(pady=10)

root.mainloop()